

# Project: Investigation of FBI GUN-CHECK 

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

INTRODUCTION

This project is about the analysis of FBI Gun-check, The Data sources are from FBI's National Instant Criminal Background Check System. The U.S. census data is found in a .csv file that contains several variables at the state level, and the NICS data is found in one sheet of an .xlsx file that contains the number of firearm checks by month, state, and type. we are going to explore the datasets to answer and get insight to the following Question;

what is the correlation between state and the gun purchasing order?.
what is the overall trending of gun purchase?.
what is state has the highest growth in Gun registration?.

In [1]:
# packages installation and loading of dataset

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px



<a id='wrangling'></a>
## Data Wrangling


In [ ]:
# Load your data and print out a few lines. Perform operations to inspect data
#   types and look for instances of missing or possibly errant data.

df_gun = pd.read_excel(r'C:\Users\k2k\Desktop\udacity alx projects\project 1\gun_data.xlsx')
df_census = pd.read_csv(r'C:\Users\k2k\Desktop\udacity alx projects\project 1\U.S. Census Data.csv')



In [ ]:
#let check the top 5 row of df_gun
df_gun.head()

In [ ]:
#let check top 5 row of df_census

df_census.head()


### Data Cleaning;

In [ ]:
#checking for duplicate in df_gun

df_gun.duplicated().sum()

In [ ]:
#checking form missing value
df_gun.dropna().head()

In [ ]:
#assigning correct data types
df_gun['month']= pd.to_datetime(df_gun['month'])

In [ ]:
df_census.head()

In [ ]:
df_census = df_census[:64]

df_census

In [ ]:
df_census.rename(columns=df_census.iloc[0]).drop(df_census.index[0])
df_census.head()

In [ ]:
#transposing df_census
df_census.set_index('Fact', inplace=True)
df_census = df_census.transpose().reset_index()

In [ ]:
df_census.info()

In [ ]:
#dropping the first row (fact_note)
df_census = df_census.drop([0])

In [ ]:
#dropping unwanted columns in df_census
df_census  = df_census.drop(df_census.iloc[:,65:], axis = 1)

In [ ]:
#checking for duplicate in  df_census

df_census.duplicated().sum()

In [ ]:
#checking for missing values in df_census

df_census.isnull().sum()

In [ ]:
#removing %,$ and comma from the data set

col = df_census.columns

df_census[col] = df_census[col].replace({'\$': '', ',': '','\%':''}, regex=True)



<a id='eda'></a>
## Exploratory Data Analysis


### Research Question 1 (What is the overall gun purchase?)

In [ ]:
#creating a purchase_trend dataframe.

purchase_trend = pd.pivot_table(data = df_gun, index = 'month', columns= 'state', values= 'totals')

purchase_trend =  purchase_trend.reset_index()

purchase_trend['total'] = purchase_trend.iloc[:,1:].sum(axis = 1)

purchase_trend

In [ ]:
#plotting total purchase trend line graph

plt.figure(figsize=(40,20))


# Preparing the data to subplots
x = purchase_trend['month']
y = purchase_trend['total']

# Plot the subplots

plt.subplot(2, 2, 1)
plt.plot(x, y, 'g')

# set chart title
plt.title("total purchase trend" , fontsize=20)

#set Axis label
plt.xlabel("Year", fontsize=20)
plt.ylabel("total purchase", fontsize=20)

plt.show()


In [ ]:
df_gun

### Research Question 2 ; Which states have had the highest growth in gun registrations?


In [ ]:
#creating a new data set form df_gun 


df_gun_growth = df_gun.loc[:, ['month', 'state', 'totals']]

df_gun_growth.reset_index()

In [ ]:
#extracting year from the month column
df_gun_growth['year']= pd.to_datetime(df_gun_growth['month']).dt.strftime('%Y')
df_gun_growth['months']= pd.to_datetime(df_gun_growth['month']).dt.strftime('%m')

#dropping the month (date column)

df_gun_growth = df_gun_growth.drop(columns = 'month')


In [ ]:
#extracting data for 1998 and 2017

df_gun_1998 = df_gun_growth.query('year == "1998"')

df_gun_2017 = df_gun_growth.query('year == "2017"')


In [ ]:
#grouping by df_gun_1998 state


df_gun_1998 = df_gun_1998.groupby(['state'])['totals'].sum().reset_index()

#rename totals as 2017totals 

df_gun_1998 = df_gun_1998.rename(columns = {'totals': '1998totals'})


In [ ]:
#grouping by df_gun_1998 state


df_gun_2017 = df_gun_2017.groupby(['state'])['totals'].sum().reset_index()
#rename totals as 2017totals 
df_gun_2017 = df_gun_2017.rename(columns = {'totals': '2017totals'})


In [ ]:
#merging th two dataset

df_growth = df_gun_2017.merge(df_gun_1998, how = 'inner', on = 'state')

df_growth

In [ ]:
#adding growth rate

df_growth['growth_rate'] = (df_growth['2017totals']- df_growth['1998totals']) / df_growth['1998totals']

df_growth = df_growth.sort_values(['growth_rate'], ascending = False).reset_index()

#removing the umwanted rows
df_growth = df_growth.drop(labels = [0,1,2], axis = 0)
df_growth= df_growth.drop(columns = {'index'}, axis= 0).reset_index()

df_growth

In [ ]:
df_growth.query('state == "Guam"')

In [ ]:

plt.figure(figsize=(20,10))
def growth_rate():
    return (df_growth['2017totals']- df_growth['1998totals']) / df_growth['1998totals']

plt.bar(df_growth['state'].head(10), df_growth['growth_rate'].head(10))
plt.ylabel("% of growth", fontsize=20)
plt.xlabel("State", fontsize=20)
# set chart title
plt.title("Top Ten highest growth in gun registration by state ", fontsize=20)

 As seen in the chart above, Guam has the highest growth in gun registration follow by kentucy and Massacusetts.

### Research Question 2 :What census data is most associated with high gun per capital?

#### Answer ; 

The cenusus data  provide data for only 2010 and 2016, so i will extract the data for 2010 and 2016 from df_gun data, then merge with census data using state as the foriegn key to get correlation for the data.


In [ ]:
#creating a dfnew_gun
df_new_gun= df_gun.loc[:, ['month', 'state', 'totals']]

df_new_gun



In [ ]:
#extracting year and month from month column

df_new_gun['year']= pd.to_datetime(df_new_gun['month']).dt.strftime('%Y')
df_new_gun['months']= pd.to_datetime(df_new_gun['month']).dt.strftime('%m')

#dropping the month (date column)

df_new_gun = df_new_gun.drop(columns = 'month')


In [ ]:
#extraction 2010-2016 data


df_new_gun = df_new_gun.query('year  == "2010"'and 'year == "2016"')


In [ ]:
#grouping df_new_gun by state
df_new_gun = df_new_gun.groupby('state')['totals'].sum().reset_index()
df_new_gun.head()

In [ ]:
#extracting census data for 2016 and 2010

#renameing the column as state

df_census_2016_2010 = df_census.iloc[:,0:4]

df_census_2016_2010.rename(columns = {'index':'state','Population estimates, July 1, 2016,  (V2016)' :'population_2016', 'Population estimates base, April 1, 2010,  (V2016)': 'population_2010' , 'Population, percent change - April 1, 2010 (estimates base) to July 1, 2016,  (V2016)': '%population_change2010and2016'}, inplace = True)


df_census_2016_2010.head()

In [ ]:
#merging the df_new_gun and df_census

df = df_census_2016_2010.merge(df_new_gun, how = 'inner', on = 'state')

df.info()

In [ ]:
#changing the datatype to float
df['population_2016'] = df['population_2016'].astype(float)
df['population_2010'] = df['population_2010'].astype(float)
df['%population_change2010and2016'] = df['%population_change2010and2016'].astype(float)


In [ ]:
#creating correlation table

#df_corr = df.corr()[['totals']][:-1].sort_values(by='totals', ascending = True)
df_corr = df.corr()
df_corr

In [ ]:
#plotting the correlation 

hm = px.imshow(df_corr, height = 1000, text_auto=True, aspect="auto")
hm.update_layout(title = 'Heatmap of census data correlation with firearm registrations', title_x = 1.0)
hm.update_xaxes(side="top")
hm.show()

<a id='conclusions'></a>
## Conclusions

> **what is the overall trend in gun purchase?**:
  > The result shows that there is increase in gun purchase over time from 1998 to 2017,the trend is seasonal which peak at december.

> **what state has the highest growth in gun purchase?**: The result shows that Guam has the highest growth (151.444444%) in Gun registration, the total gun registration increase from  9 in 1998 to 1372 in 2017.
>

> **What census data is most associated with high gun per capital?**: Both 2010 population and 2016 population census data have positive correlation with the total gun checkedin 2010 and 2016. 2010 population census has 0.605044 which is greater than 0.605044 of 2016 population census.


## Limitation
>The main limitation is the lack of census data that prevented more detailed analysis: the census data is available only for the period 2010 - 2016. Missing data in the NICS  Gun-check dataset were removed.
